<a href="https://colab.research.google.com/github/seenu-g/eva4-2/blob/master/week9/nlp6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** Transformers for Sentiment Analysis**

In [1]:
pip install transformers

     |████████████████████████████████| 1.4MB 13.3MB/s 
     |████████████████████████████████| 890kB 34.7MB/s 
     |████████████████████████████████| 2.9MB 47.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a83b4c8e472db01f0d8514a43b392e3ddcde16287ace6fef4a1487a24f491e2b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

transformer has already been trained with a specific vocabulary, which means we need to train with the exact same vocabulary and also tokenize our data in the same way that the transformer did when it was initially trained.transformers library has tokenizers for each of the transformer models provided

In [3]:
# we are using the BERT model which ignores casing (i.e. will lower case every word)
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
# The tokenizer has a vocab attribute which contains the actual vocabulary and check how many tokens are in it by checking its length.
len(tokenizer.vocab)

30522

In [5]:
# tokenize and lower case the data in a way that is consistent with the pre-trained transformer model.
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')
print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [6]:
# numericalize tokens using our vocabulary using tokenizer.convert_tokens_to_ids
indexes = tokenizer.convert_tokens_to_ids(tokens)
print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [7]:
# tokenizer does have a beginning of sequence and end of sequence attributes (bos_token and eos_token)
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [8]:
# get the indexes of the special tokens by converting them using the vocabulary
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)
print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [9]:
# explicitly getting them from the tokenizer.
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id
print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [10]:
# get the maximum length of these input sizes by checking the max_model_input_sizes for the version of the transformer we want to use
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

512


In [11]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [12]:
from torchtext import data
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [13]:
# Load the data and create the validation splits
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:09<00:00, 9.24MB/s]


Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [14]:
# Take example and ensure that the text has already been numericalized
print(vars(train_data.examples[6]))

{'text': [1997, 2607, 1045, 2001, 3666, 1038, 2290, 1012, 1045, 3866, 1055, 2487, 1010, 1045, 4669, 2161, 1016, 1010, 2161, 1017, 2001, 7929, 1010, 1998, 3866, 1996, 2345, 2028, 1012, 8038, 2100, 1010, 2045, 2003, 1037, 6714, 1011, 2125, 2265, 999, 1045, 2134, 1005, 1056, 2113, 2055, 2023, 2012, 2035, 1010, 2028, 1997, 2026, 2814, 2409, 2033, 2055, 2023, 1012, 1045, 2001, 2428, 7568, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1045, 3427, 1996, 2034, 1017, 4178, 1012, 1012, 1012, 2054, 1037, 3538, 1997, 29132, 999, 9454, 2611, 3689, 17037, 1012, 2045, 2003, 2053, 2671, 4349, 1012, 1012, 1012, 2092, 1010, 6684, 2151, 1012, 2012, 1996, 2203, 1997, 2296, 2792, 2057, 2064, 4608, 1037, 12185, 1997, 1037, 22330, 7811, 1012, 2008, 1005, 1055, 2035, 1012, 2040, 14977, 2023, 1029, 2106, 2027, 5630, 2008, 1996, 2279, 2265, 1005, 1055, 4539, 4378, 2097, 2022, 3801, 2104, 2324, 1029, 11771, 3412, 14652, 3331, 1010, 22653, 1010, 7743, 2075, 1012, 1012, 1012, 1998, 2070, 2062, 1012, 1026, 

In [15]:
# use the convert_ids_to_tokens to transform these indexes back into readable tokens.
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])
print(tokens)

['of', 'course', 'i', 'was', 'watching', 'b', '##g', '.', 'i', 'loved', 's', '##1', ',', 'i', 'liked', 'season', '2', ',', 'season', '3', 'was', 'ok', ',', 'and', 'loved', 'the', 'final', 'one', '.', 'ya', '##y', ',', 'there', 'is', 'a', 'spin', '-', 'off', 'show', '!', 'i', 'didn', "'", 't', 'know', 'about', 'this', 'at', 'all', ',', 'one', 'of', 'my', 'friends', 'told', 'me', 'about', 'this', '.', 'i', 'was', 'really', 'excited', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'i', 'watched', 'the', 'first', '3', 'episodes', '.', '.', '.', 'what', 'a', 'piece', 'of', 'rubbish', '!', 'teenage', 'girl', 'drama', 'fest', '.', 'there', 'is', 'no', 'science', 'fiction', '.', '.', '.', 'well', ',', 'hardly', 'any', '.', 'at', 'the', 'end', 'of', 'every', 'episode', 'we', 'can', 'catch', 'a', 'glimpse', 'of', 'a', 'cy', '##lon', '.', 'that', "'", 's', 'all', '.', 'who', 'cares', 'this', '?', 'did', 'they', 'decide', 'that', 'the', 'next', 'show', "'", 's', 'target', 'audience', 'will', 'be',

In [16]:
# we still need to build the vocabulary for the labels.
LABEL.build_vocab(train_data)

In [17]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f9e4bab38c8>, {'neg': 0, 'pos': 1})


In [18]:
# use the largest batch size that we can to get best results for transformers.
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [19]:
# load the pre-trained model,
from transformers import BertTokenizer, BertModel
bert = BertModel.from_pretrained('bert-base-uncased')

In [20]:
import torch.nn as nn
# Instead of using an embedding layer to get embeddings for our text, we'll be using the pre-trained transformer model. 
# These embeddings will then be fed into a GRU to produce a prediction for the sentiment of the input sentence. 
# We get the embedding dimension size (called the hidden_size) from the transformer via its config attribute.
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()    
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]       
        with torch.no_grad():
            embedded = self.bert(text)[0]
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        #hidden = [n layers * n directions, batch size, emb dim]
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        #output = [batch size, out dim]
        return output

In [21]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [22]:
# check how many parameters the model 
#  Our standard models have under 5M, but this one has 112M! 
#  Luckily, 110M of these parameters are from the transformer and we will not be training those.
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [23]:
# In order to freeze paramers (not train them) we need to set their requires_grad attribute to False. 
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [24]:
#  double check the names of the trainable parameters, ensuring they make sense. 
# they are all the parameters of the GRU (rnn) and the linear layer (out)
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [25]:
# we define our optimizer and criterion (loss function).
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
# Place the model and criterion onto the GPU (if available)
model = model.to(device)
criterion = criterion.to(device)

In [26]:
# calculating accuracy
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc
  
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0  
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# calculating how long a training/evaluation epoch takes.
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [27]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 7m 5s
	Train Loss: 0.518 | Train Acc: 73.19%
	 Val. Loss: 0.285 |  Val. Acc: 88.56%
Epoch: 02 | Epoch Time: 7m 5s
	Train Loss: 0.290 | Train Acc: 88.19%
	 Val. Loss: 0.238 |  Val. Acc: 90.39%
Epoch: 03 | Epoch Time: 7m 5s
	Train Loss: 0.241 | Train Acc: 90.45%
	 Val. Loss: 0.252 |  Val. Acc: 89.63%
Epoch: 04 | Epoch Time: 7m 5s
	Train Loss: 0.212 | Train Acc: 91.77%
	 Val. Loss: 0.217 |  Val. Acc: 91.65%
Epoch: 05 | Epoch Time: 7m 6s
	Train Loss: 0.188 | Train Acc: 92.70%
	 Val. Loss: 0.206 |  Val. Acc: 92.28%


In [28]:
# load up  parameters that gave us  best validation loss and try  on the test set 
model.load_state_dict(torch.load('tut6-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.197 | Test Acc: 92.23%


In [32]:
# use  model to test sentiment of some sequences. We tokenize input sequence, trim it down to te maximum length, 
# add  special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [33]:
predict_sentiment(model, tokenizer, "This film is terrible")


0.02194758877158165

In [34]:
predict_sentiment(model, tokenizer, "This film is great")


0.9530491232872009